# Advection Diffusion Equation

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np
import matplotlib.pyplot as plt

import time
torch.set_default_dtype(torch.float64)

In [2]:
def u_true(x):
    return np.sin(x[:,0]) * np.exp(-x[:,1])

def f(x):
    return np.cos(x[:,0]) * np.exp(x[:,1])


def g(x):
    return u_true(x)

def h(x):
    return np.sin(x[:,0])

In [3]:
def samples(m_train, m_bd, m_int):
    
    # interior points and time
    x_train = np.random.uniform(-1, 1, size=(m_train, 1))
    t_train = np.random.randn(m_train, 1)
    inputs = np.concatenate((x_train,t_train), axis=1)
    
    
    # boundary points
    # left boundary
    t_left = np.random.rand(m_bd, 1)
    x_left = - np.ones(t_left.shape)
    left = np.concatenate((x_left,t_left), axis=1)
    # right boundary
    t_right = np.random.rand(m_bd, 1)
    x_right = np.ones(t_right.shape)
    right = np.concatenate((x_right, t_right), axis=1)
    bd = np.concatenate((left,right))
    
    # initial points
    x_int = np.random.uniform(-1, 1, size=(m_int, 1))
    t_int = np.zeros(x_int.shape)
    initial = np.concatenate((x_int, t_int), axis=1)
    
    return inputs, bd, initial

In [4]:
# Generate training samples
# number of interior points
m_train = 1000    
# number of boundary points
m_bd = 100
# number of initial points
m_int = 200

x_inputs, x_bd, x_initial = samples(m_train, m_bd, m_int)


# test at random points
m_test = 1024
x_test = np.random.uniform(-1,1,size=(m_test,1) )
t_test = np.random.rand(m_test, 1)
test = np.concatenate((x_test,t_test), axis=1)
u_truth = u_true(test)

# generate random features
N = 1000
sigma = 3
W = np.random.randn(2,N) / sigma
b = np.random.uniform(0,2*np.pi, size=(N,))

# generate random feature matrices
A1 = np.cos(x_inputs @ W + b)
A2 = np.sin(x_inputs @ W + b)

# generate left matrix
B1 = - A2 * W[1] + A1 * (W[0] ** 2) - A2 * W[0]

A = np.row_stack( (B1, np.cos(x_bd @ W + b), np.cos(x_initial @ W + b) ) )
y = np.concatenate( (f(x_inputs), g(x_bd), h(x_initial)) )   

#c = np.linalg.solve(A.T @ A + 1e-3 * np.eye(N), A.T@y)
start = time.time()
c = np.linalg.pinv(A) @ y
end = time.time()
# 
u_test = np.cos(test @ W + b) @ c

err = np.mean( (u_truth - u_test)**2 )
print(err)
print(f'{end-start:.2f} seconds')

0.05697370622639651
0.01 seconds


In [5]:
# generate random features
N = 1000
sigma1 = 20
sigma2 = 10000
W1 = np.random.randn(1,N) / sigma1
W2 = np.random.randn(1,N) / sigma2
W = np.concatenate((W1,W2))
b = np.random.uniform(0,2*np.pi, size=(N,))

# generate random feature matrices
A1 = np.cos(x_inputs @ W + b)
A2 = np.sin(x_inputs @ W + b)

# generate left matrix
B1 = - A2 * W2 + A1 * (W1 ** 2) - A2 * W1

A = np.row_stack( (B1, np.cos(x_bd @ W + b), np.cos(x_initial @ W + b) ) )
y = np.concatenate( (f(x_inputs), g(x_bd), h(x_initial)) )   

#c = np.linalg.solve(A.T @ A + 1e-3 * np.eye(N), A.T@y)
start = time.time()
c = np.linalg.pinv(A) @ y
end = time.time()

# 
u_test = np.cos(test @ W + b) @ c

err = np.mean( (u_truth - u_test)**2 )
print(err)
print(f'{end-start:.2f} seconds')

0.03376980438901909
0.21 seconds


In [6]:
# ELM

# generate random features
N = 1000
R = 5
W = np.random.uniform(-R,R,size=(2,N))

# generate random feature matrices
A1 = np.tanh(x_inputs @ W)

# generate left matrix
B1 = (1 - A1**2) * (W[1] + W[0]) + 2 * A1 * (1-A1**2) * (W[0] ** 2)

A = np.row_stack( (B1, np.tanh(x_bd @ W), np.tanh(x_initial @ W) ) )
y = np.concatenate( (f(x_inputs), g(x_bd), h(x_initial)) )   

#c = np.linalg.solve(A.T @ A + 1e-3 * np.eye(N), A.T@y)
start = time.time()
c = np.linalg.pinv(A) @ y
end = time.time()

# 
u_test = np.tanh(test @ W) @ c

err = np.mean( (u_truth - u_test)**2 )
print(err)
print(f'{end-start:.2f} seconds')

0.035288078510435174
0.23 seconds
